# <font color=#cd0000> Propósito principal </font>
- La idea de este librillo es preparar el entorno para realizar pruebas para cualquier DataSet a ser clasificado por cualquier codificación aplicada a LSTM

## <font color=#cd0000> Leyenda </font>
- Los apartados titulados con el código de colores HEX: `#cd0000` serán apartados que tendrán todos los librillos, en concreto, aquellos especificados en el apartado `Síntesis de los criterios usados` del trabajo.
- Los apartados titulados con el código de colores HEX: `#2451ff` serán apartados de conclusiones propias de este librillo resultado de aplicar un estudio personalizado para cada planteamiento.

# <font color=#cd0000> Prerrequisitos </font>
## <font color=#cd0000> Entorno de ejecución </font>
- Cambiamos el directorio raíz del librillo para acceder cómodamente a las funciones de utilidad.

In [ ]:
import os

os.chdir('../..')

## <font color=#cd0000> Constantes y variables predefinidas </font>
- TODO -> Añadir SEED a todas las particiones.

In [ ]:
HEARTBEAT_PATH = "data/heartbeat"
EPILEPSY_PATH = "data/epilepsy"
SEGUIMIENTO_OCULAR_PATH = "data/seguimiento-ocular/Data/Hospital"
SEGUIMIENTO_OCULAR_FOLDERS_ID = range(1, 12+1)

SEED = 1

# <font color=#cd0000> Carga del Dataset </font>
- TODO: Breve descripción

In [ ]:
# TODO - Change with known data
from utils.load_data import import_epilepsy_dataset

train, test = import_epilepsy_dataset(EPILEPSY_PATH)


# <font color=#cd0000> Preprocesamiento </font>

## <font color=#cd0000> Eliminación de datos inválidos y valores atípicos </font>
- TODO: Breve descripción de qué es un dato inválido (-1's en columna, etc.)
- Eliminaremos aquellos valores fuera de los percentiles 5 y 95.
- TODO: Definiremos cuál será el límite de outliers permitido por serie temporal

In [ ]:
# TODO - Remove invalid data

In [ ]:
# train.remove_outliers(headers=train.get_derived_data_columns()['attrs'], outliers_limit=0.9)
# test.remove_outliers(headers=test.get_derived_data_columns()['attrs']) -> Estos no los podemos alterar

In [ ]:
import pandas as pd

# Remaining series
print("Previous number of series: {}".format(
    len(pd.unique(train.original_data['id']))))
print("Actual number of series: {}".format(
    len(pd.unique(train.derived_data['id']))))


## <font color=#cd0000> Normalización </font>

In [ ]:
from utils.codifications import standardize_data

train.derived_data, test.derived_data = standardize_data(
    train.derived_data, test.derived_data, headers=train.get_derived_data_columns()['attrs'])

In [ ]:
train.derived_data

## <font color=#cd0000> Resoluciones a aplicar </font>
- TODO:
  - Si las series son rápidas (muchos cambios en poco tiempo) especificar resoluciones altas (sin modificaciones).
  - Si las series son lentas (pocos cambios en mucho tiempo) especificar resoluciones bajas (eliminamos datos).

In [ ]:
# Series lentas
train.reduce_sampling_rate(remove_one_each_n_samples=2)
test.reduce_sampling_rate(remove_one_each_n_samples=2)

In [ ]:
train.derived_data

# <font color=#cd0000> Codificación </font>
- TODO: Breve descripción de la codificación

In [ ]:
from utils.codifications import temporal_trend_fn

train.apply_codifications([temporal_trend_fn])
test.apply_codifications([temporal_trend_fn])

train.derived_data

In [ ]:
X_train, _ = train.transform_derived_data_into_X_y()
X_test, _ = test.transform_derived_data_into_X_y()

y_train = train.derived_data.groupby('id').first()['class'].to_numpy()
y_test = test.derived_data.groupby('id').first()['class'].to_numpy()

## <font color=#cd0000> Tensores entrada y salida de la red </font>

In [ ]:
# All lengths must be equal
series_length = train.get_shortest_serie().shape[0]
n_dims = len(train.get_derived_data_columns()['attrs'])

(
    train.get_shortest_serie().shape[0],
    train.get_largest_serie().shape[0],
    test.get_shortest_serie().shape[0],
    test.get_largest_serie().shape[0]
)


In [ ]:
import pandas as pd

# This will determine the number of series of each split
train_n_series = pd.unique(train.derived_data['id']).shape[0]
test_n_series = pd.unique(test.derived_data['id']).shape[0]

(train_n_series, test_n_series)


In [ ]:
from utils.classifier_utils import apply_lstm_format
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder()
enc.fit(y_train.reshape(-1, 1))

X_train, y_train = apply_lstm_format(
    X_train, y_train, train_n_series, series_length, 2, enc)
X_test, y_test = apply_lstm_format(
    X_test, y_test, test_n_series, series_length, 2, enc)

# <font color=#cd0000> Diseño de la topología de red </font>
- TODO:
  - Nº unidades LSTM (rangos a variar)
  - Funciones de activación
  - Tamaño por lote (batch_size)
  - Neuronas de salida
    - Una con activación sigmoidal si es binaria
    - Tantas como clases haya con activación softmax + argmax para decidir qué clase será la elegida.
      - Para hacer esto -> `k.argmax`
  - ...

In [ ]:
import keras as k
from keras.models import Sequential
from keras.layers import LSTM, Dense

nn = Sequential()

# Number of initial dimensions
nn.add(LSTM(units=12, dropout=.2, recurrent_dropout=.2))
# Number of Epilepsy's classes
nn.add(Dense(4, activation='sigmoid'))

## <font color=#cd0000> Compilación de la red </font>
- TODO:
  - Optimizador -> reajuste de pesos
    - Función de pérdida
  - Función de pérdida
  - Métricas

In [ ]:
from keras.optimizers import RMSprop

metrics = [
    k.metrics.CategoricalAccuracy(name="ACC"),
    k.metrics.Precision(name='Prec'),
    k.metrics.Recall(name='Rec'),
    k.metrics.AUC(name='AUC')
]
nn.compile(optimizer=RMSprop(learning_rate=1e-4), loss='categorical_crossentropy', metrics=metrics)
nn.build(input_shape=X_train.shape)

## <font color=#cd0000> Visualización de resultados preliminares </font>
- TODO:
  - Verificar sobreajuste y tomar medidas en caso de darse:
    - Regularizadores L1L2
    - Tasa de Dropout
    - Decrementar épocas de entrenamiento
  - Si tarda en converger:
    - Inicializar correctamente los pesos (GlorotNormal, ...)

In [ ]:
import matplotlib.pyplot as plt


def show_metrics(history):
    for metric in history.history.keys():
        if not metric.startswith('val_'):
            plt.plot(history.history[metric], label=metric)
            plt.plot(history.history[f'val_{metric}'], label=f'val_{metric}')
            plt.title(metric)
            plt.ylabel('')
            plt.xlabel('Epoch')
            plt.legend(loc="upper left")
            plt.show()


# <font color=#cd0000> Entrenamiento del modelo </font>

In [ ]:
import numpy as np
import keras.backend as K


def argmax(x, n_classes):
    all_predictions = []
    for max_class_position in K.argmax(x):
        prediction = np.zeros(n_classes)
        prediction.put(max_class_position, 1)
        all_predictions.append(prediction)
    return np.asarray(all_predictions)


In [ ]:
import numpy as np
from keras import backend as K
from sklearn.metrics import confusion_matrix, classification_report

epochs = 10

history = nn.fit(X_train, y_train, epochs=epochs,
                 validation_data=(X_test, y_test))
nn.summary()
print('\n\n')

y_pred = argmax(nn.predict(X_test), 4)

show_metrics(history)

In [ ]:
y_real = enc.inverse_transform(y_test)
y_pred = enc.inverse_transform(y_pred)
print(confusion_matrix(y_real, y_pred))
print(classification_report(y_real, y_pred, zero_division=0))

# <font color=#cd0000> Randomized Search </font>
- Búsqueda de hiper-parámetros aleatoria con LSTM maximizando ``macro avg f1-score``

## <font color=#cd0000> Randomized Search + CV </font>
- Solo si tenemos muchos datos

In [ ]:
import random
import pickle
import utils.constants as cs
from utils.classifier_utils import (windowed_cross_val,
                                    compute_classification_reports_means)
from utils.plot_utils import pretty_print_classification_report_dict
from keras.layers import LSTM, Dense
from keras.optimizers import RMSprop

PKL_DIR = 'pkl/LSTM/'


def lstm_randomized_search_cv(
        windowed_series,
        relation_with_series,
        prefix,
        cv=5,
        lstm_dict={}):
    global PKL_DIR
    all_clf_used = {}

    n_samples = 5
    units_list = random.sample(
        [30, 50, 75, 100, 200], n_samples)
    learning_rate_list = random.sample(
        [1e-2, 1e-3, 1e-4, 1e-5, 1e-6], n_samples)

    best_hyp_params = None
    best_score = 0
    for units in units_list:
        for learning_rate in learning_rate_list:
            clf_used = {}

            lstm_dict[cs.LSTM_HYP_PARAM_UNITS] = units
            lstm_dict[cs.LSTM_HYP_PARAM_LEARNING_RATE] = learning_rate
            reports = windowed_cross_val(
                None,
                windowed_series,
                relation_with_series,
                cv=cv,
                seed=SEED,
                drop_columns=['class'],
                estimator_type=cs.ESTIMATOR_LSTM,
                lstm_dict=lstm_dict
            )

            mean_report = compute_classification_reports_means(reports)
            all_clf_used[(units, learning_rate)] = (clf_used, str(mean_report))

            if mean_report['macro avg']['f1-score'] >= best_score:
                best_score = mean_report['macro avg']['f1-score']
                best_hyp_params = (units, learning_rate)
                best_report = mean_report

            print("\t\t--------------ACTUAL BEST: Units={}; Learning Rate={}--------------"
                  .format(best_hyp_params[0], best_hyp_params[1]))
            pretty_print_classification_report_dict(best_report)
            print("\t\t--------------ITERATION: Units={}; Learning Rate={}--------------"
                  .format(units, learning_rate))
            pretty_print_classification_report_dict(mean_report)

    with open(PKL_DIR + prefix, 'wb') as file:
        pickle.dump(all_clf_used, file)

    return best_hyp_params, best_report


In [ ]:
lstm_dict = {
    cs.LSTM_SERIES_LENGTH: series_length,
    cs.LSTM_SEQUENCES_FRAGMENTER: 2,
    cs.LSTM_FITTED_LABELS_ENCODER: enc,
    cs.LSTM_ARGMAX_FUNCTION: argmax,
    cs.LSTM_N_CLASSES: 4,
    cs.LSTM_HYP_PARAM_EPOCHS: 50
}

lstm_randomized_search_cv(
    train.derived_data,
    train.derived_data_windows_per_serie,
    'lstm_sample',
    cv=5,
    lstm_dict=lstm_dict
)


# <font color=#cd0000> Randomized Search con múltiples ejecuciones en lugar de Validación Cruzada </font>
- Solo si tenemos pocos datos
- Ejecutaremos el mismo modelo sobre diferentes particiones del conjunto de datos original para observar su desempeño.

In [ ]:
# TODO

# <font color=#cd0000> Análisis de resultados </font>
- TODO - Un breve análisis de los resultados obtenidos para las diferentes resoluciones, ventanas, ...
- Visualización de gráficos para determinar si se pueden obtener mejores resultados con una serie de hiper-parámetros concretos.

In [ ]:
# TODO

# <font color=#cd0000> Conclusiones </font>
- TODO - Unas breves conclusiones sobre los resultados obtenidos (influencia de la codificación, ...)